# <center>Home Assignment 1</center>
### <center>Youssef Benhachem</center>


### **(5) -** Logitic regression implementation:

In [ ]:
import numpy as np
from tqdm import tqdm
from PIL import Image
import os
from tqdm import tqdm
import random

In [ ]:
class LogisticRegression:
    """Logistic regression with SGD as the solver.

    Example usage:
        > clf = LogisticRegression()
        > clf.fit(x_train, y_train)
        > clf.predict(x_eval)
    """
    def __init__(self, learning_rate=0.001, max_iter=1000, theta_0=None):
        """
        Args:
            learning_rate: Learning rate for sgd
            max_iter: Maximum number of iterations for the solver.
            theta_0: Initial guess for theta. If None, use the zero vector.
        """
        self.theta = theta_0
        self.learning_rate = learning_rate
        self.max_iter = max_iter
    
    def fit(self, x, y):
        """Run SGD to minimize the logistic loss for logistic regression.

        Args:
            x: Training example inputs. Shape (n_examples, dim).
            y: Training example labels. Shape (n_examples,).
        """
        for _ in tqdm(range(self.max_iter)):
            for obs, label in zip(x, y):
                self.theta += self.learning_rate * label * (1/(1+np.exp(label*np.dot(self.theta,obs)))) * obs
    
    def fit_reg_l2(self, x, y, lam):
        """Run SGD with l2-regularization to minimize the logistic loss for logistic regression.

        Args:
            x: Training example inputs. Shape (n_examples, dim).
            y: Training example labels. Shape (n_examples,).
        """
        for _ in tqdm(range(self.max_iter)):
            for obs, label in zip(x, y):
                self.theta = (1 - 2* lam* self.learning_rate)* self.theta + self.learning_rate * label * (1/(1+np.exp(label*np.dot(self.theta,obs)))) * obs
    
    def fit_reg_l1(self, x, y, lam):
        """Run SGD with l1-regularization to minimize the logistic loss for logistic regression.

        Args:
            x: Training example inputs. Shape (n_examples, dim).
            y: Training example labels. Shape (n_examples,).
        """
        for _ in tqdm(range(self.max_iter)):
            for obs, label in zip(x, y):
                self.theta = self.theta -self.learning_rate*lam*np.sign(self.theta) + self.learning_rate * label * (1/(1+np.exp(label*np.dot(self.theta,obs)))) * obs
                
                
    def predict(self, x):
        """Return predicted probabilities given new inputs x.

        Args:
            x: Inputs of shape (n_examples, dim).

        Returns:
            Outputs of shape (n_examples,).
        """
        return 1/(1+np.exp(-np.dot(x, self.theta)))
    
    def evaluate(self, x_test, y_test):
        y_pred = self.predict(x_test)
        loss = np.sum(np.log(1+np.exp(-y_test*np.dot(x_test, self.theta))))
        return loss/len(y_pred)
    
    def zero_one_loss(self, x_test, y_test):
        y_pred = self.predict(x_test)
        loss = np.sum(abs(y_pred - y_test))
        return loss/len(y_pred)

In [ ]:
def load_train():
    train_data = []
    letter2output = {'A':1,'B':-1, 'C':-1}
    
    for letter in tqdm(['A','B','C']):
        train_path = f'data/train/{letter}'
        images_names = (os.listdir(train_path))
        for img_path in (f'{train_path}/{img_name}' for img_name in images_names):
            img = Image.open(img_path)
            flat = np.array(img).flatten()
            img_vec = np.append(flat/np.linalg.norm(flat), letter2output[letter])
            train_data.append(img_vec)
    return np.array(train_data)

def load_test():
    test_data = []
    letter2output = {'A':1,'B':-1, 'C':-1}
    
    for letter in tqdm(['A','B','C']):
        test_path = f'data/test/{letter}'
        images_names = (os.listdir(test_path))
        for img_path in (f'{test_path}/{img_name}' for img_name in images_names):
            img = Image.open(img_path)
            flat = np.array(img).flatten()
            img_vec = np.append(flat/np.linalg.norm(flat), letter2output[letter])
            test_data.append(img_vec)
    return np.array(test_data)

In [ ]:
# Loading the data
train_data =load_train()
test_data = load_test()

# Randomizing the data
random.shuffle(train_data)
random.shuffle(test_data)

In [ ]:
# Adding the intercept for Linear Models
train_data = np.c_[np.ones(6000), train_data]
test_data = np.c_[np.ones(750), test_data]


In [ ]:
# Importing train and test data
x_train = train_data[:, :-1]
y_train = train_data[:, -1]
x_test = test_data[:, :-1]
y_test = test_data[:, -1]

**(5)-(a)-**

In [ ]:
import random as rd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def train_test_loss(learning_rate, iterations, x_train, y_train, x_test, y_test, classifier):
    """
    
        Args:
            learning_rate: Learning rate for sgd
            iterations : List representing the maximum iterations to fit our model
    """
    train_loss = []
    test_loss = []
    
    for it in iterations: 
        if classifier == "LogisticRegression":
            clf = LogisticRegression(theta_0 = np.zeros(785), max_iter = it, learning_rate=learning_rate)
        elif classifier == "LinearRegression":
            clf = LinearRegression(theta_0 = np.zeros(785), max_iter = it, learning_rate=learning_rate)
        elif classifier == "Perceptron":
            clf = Perceptron(theta_0 = np.zeros(785), max_iter = it, learning_rate=learning_rate)
        clf.fit(x_train, y_train)
        train_loss.append(clf.evaluate(x_train, y_train))
        test_loss.append(clf.evaluate(x_test, y_test))

    return train_loss, test_loss

def plot_loss(learning_rates, iterations, x_train, y_train, x_test, y_test, classifier):
    train_loss_curves = []
    test_loss_curves = []
    
    for learning_rate in learning_rates:
        result = train_test_loss(learning_rate, iterations, x_train, y_train, x_test, y_test, classifier)
        
        train_loss_curves.append(result[0])
        test_loss_curves.append(result[1])
        
        train_curve, test_curve = result[0], result[1]
    
    for train_curve, learning_rate in zip(train_loss_curves, learning_rates):
        #col = (np.random.random(), np.random.random(), np.random.random())
        plt.plot(iterations, train_curve, label=f"{learning_rate}")
        
    plt.title(f"{classifier}: Training Error")
    plt.xlabel("Iterations")
    plt.ylabel("Training Loss")
    plt.legend()
    plt.show()
    
        
    for test_curve, learning_rate in zip(test_loss_curves, learning_rates):
        #cola = (rd.uniform(0,1), rd.uniform(0,1), rd.uniform(0,1))
        plt.plot(iterations, test_curve, label=f"{learning_rate}")
        
    plt.title(f"{classifier} : Testing Error")
    plt.xlabel("Iterations")
    plt.ylabel("Testing Loss")
    plt.legend(loc="upper left")
    plt.show()
        
    return train_loss_curves, test_loss_curves

=> The best learning rate is then for Logistic Regression is then $0.001$ followed by $0.01$

**(5)-(b)-**

In [ ]:
def save_theta(iterations, x_train, y_train, classifier):
    thetas = []
    for it in iterations:
        if classifier == "LogisticRegression":
            clf = LogisticRegression(theta_0 = np.zeros(785), max_iter = it, learning_rate=0.001)
        elif classifier == "LinearRegression":
            clf = LinearRegression(theta_0 = np.zeros(785), max_iter = it, learning_rate=0.001)
        elif classifier == "Perceptron":
            clf = Perceptron(theta_0 = np.zeros(785), max_iter = it, learning_rate=0.001)
        elif classifier == "KNN":
            clf = KNN(5, x_train,y_train)
            
        clf.fit(x_train, y_train)
        thetas.append(clf.theta)
    return thetas


In [ ]:
thetas = save_theta([10,100,1000,10000], x_train, y_train, classifier="LogisticRegression")

In [ ]:
from PIL import Image
from matplotlib import cm
def show_thetas(thetas):
    for theta in thetas:
        theta = np.abs(theta/np.linalg.norm(theta))
        theta = theta[1:].reshape(28,28)
        im = Image.fromarray(np.uint8(cm.gist_earth(theta)*255))
        scaled_img = im.resize((250, 250))
        display(scaled_img)

In [ ]:
show_thetas(thetas)

### Linear Least Squares Regression

In [ ]:
import numpy as np
from tqdm import tqdm

In [ ]:
class LinearRegression:
    """Linear regression with SGD as the solver.

    Example usage:
        > clf = LinearRegression()
        > clf.fit(x_train, y_train)
        > clf.predict(x_eval)
    """
    def __init__(self, learning_rate=0.0001, max_iter=1000,
                 theta_0=None):
        """
        Args:
            learning_rate: Learning rate for sgd
            max_iter: Maximum number of iterations for the solver.
            theta_0: Initial guess for theta. If None, use the zero vector.
        """
        self.theta = theta_0
        self.learning_rate = learning_rate
        self.max_iter = max_iter
     
    
    def fit(self, x, y):
        """Run SGD to minimize the Squares Loss for Linear regression.

        Args:
            x: Training example inputs. Shape (n_examples, dim).
            y: Training example labels. Shape (n_examples,).
        """
        for _ in tqdm(range(self.max_iter)):
            for obs, label in zip(x, y):
                self.theta -= 2 * self.learning_rate * (self.predict(obs) - label) * obs

    def predict(self, x):
        """Return predicted probabilities given new inputs x.

        Args:
            x: Inputs of shape (n_examples, dim).

        Returns:
            Outputs of shape (n_examples,).
        """
        return np.dot(x, self.theta)
    
    def evaluate(self, x_test, y_test):
        y_pred = np.sign(self.predict(x_test))
        loss = np.sum((y_test - y_pred)**2)
        return loss/len(y_pred)

In [ ]:
import matplotlib.pyplot as plt
resl = plot_loss([ 0.001,0.0001, 0.00001], [10, 100, 250, 500, 1000, 5000], x_train, y_train, x_test, y_test, classifier ="LinearRegression" )

=> The best learning rate for *Linear Regression* is then $\eta = 0.0001$

In [ ]:
thetas = save_theta([10,100,1000,10000], x_train, y_train, classifier="LinearRegression")

In [ ]:
show_thetas(thetas)

### Pereptron algorihtm

In [ ]:
import numpy as np
from tqdm import tqdm

In [ ]:
class Perceptron:
    """ Perceptron Algorithm

    Example usage:
        > clf = Perceptron()
        > clf.fit(x_train, y_train)
        > clf.predict(x_eval)
    """
    def __init__(self, learning_rate=0.001, max_iter=1000, theta_0=None ):
        """
        Args:
            learning_rate: Learning rate for sgd
            max_iter: Maximum number of iterations for the solver.
            theta_0: Initial guess for theta. If None, use the zero vector.
        """
        self.theta = theta_0
        self.learning_rate = learning_rate
        self.max_iter = max_iter
     
    
    def fit(self, x, y):
        """
        Args:
            x: Training example inputs. Shape (n_examples, dim).
            y: Training example labels. Shape (n_examples,).
        """
        for _ in tqdm(range(self.max_iter)):
            for obs, label in zip(x, y):
                self.theta -= self.learning_rate * (self.predict(obs) - label) * obs

    def predict(self, x):
        """Return predicted probabilities given new inputs x.

        Args:
            x: Inputs of shape (n_examples, dim).

        Returns:
            Outputs of shape (n_examples,).
        """
        if np.dot(x, self.theta)>= 0:
            prediction = 1
        else:
            prediction = -1
            
        return prediction
    
    def evaluate(self, x_test, y_test):
        y_pred = np.fromiter(map(self.predict, x_test), dtype = float)
        loss = np.sum(abs(y_test - y_pred))
        return loss/(2*len(y_pred))

In [ ]:
resp = plot_loss([0.1, 0.01, 0.001,0.0001], [10, 100, 250, 500, 1000], x_train, y_train, x_test, y_test, classifier ="Perceptron" )

=> We see that the loss of the *Perceptron* does **not depend** on the learning rate, given that all the curves are *overlapped*

In [ ]:
thetas = save_theta([10,100,1000,10000], x_train, y_train, classifier="Perceptron")
show_thetas(thetas)

### (6) - K Nearest Neighbors (KNN)

**(b)** -

In [ ]:
# Loading the data without adding the intercept 
train_data = load_train()
test_data = load_test()

# Splitting into train and test data
x_train = train_data[:, :-1]
y_train = train_data[:, -1]
x_test = test_data[:, :-1]
y_test = test_data[:, -1]

In [ ]:
import numpy as np
from tqdm import tqdm

In [ ]:
class KNN:
    """KNN algorithm

    Example usage:
        > clf = KNN(5, x_train, y_train)
        > clf.predict(x_eval)
    """
    def __init__(self, K, x_train, y_train):
        """
        Args:
            theta_0: Initial guess for theta. If None, use the zero vector.
        """
        self.K = K
        self.x_train = x_train
        self.y_train = y_train
     
    def distance(self, a, b):
        return np.linalg.norm(a-b)
    
    def fit(self, x_train, y_train):
        pass
    
    def predict(self, x):
        """Return predicted probabilities given new inputs x.

        Args:
            x: Inputs of shape (n_examples, dim).

        Returns:
            Outputs of shape (n_examples,).
        """
        distances = np.fromiter(map(lambda vec:self.distance(vec, x), self.x_train), dtype = float) 
        inds = np.argsort(distances)[:self.K]
        prop_ones = sum(self.y_train[inds])/len(inds)
        if prop_ones >= 0:
            return 1
        return -1
    
    def evaluate(self, x_test, y_test):
        y_pred = []
        for test_sample in x_test:
            y_pred.append(self.predict(test_sample))
            
        y_pred = np.array(y_pred)
        loss = np.sum(abs((y_test - y_pred)))/(2*len(y_test))
        return loss

In [ ]:
def plot_loss_KNN( x_train, y_train, x_test, y_test):
    loss = []
    for K in tqdm(range(1,50,1)):
        clf = KNN(K, x_train[1000:3000], y_train[1000:3000])
        loss.append(clf.evaluate(x_test, y_test))
    plt.plot(range(1,50,1), loss)
    plt.show()


In [ ]:
plt.title("Testing Loss")
plot_loss_KNN( x_train, y_train, x_test, y_test)

In [ ]:
import matplotlib.pyplot as plt
plt.title("Training Loss")
plot_loss_KNN( x_train, y_train, x_train, y_train)

###  (6-c)  K-fold cross-validation to calibrate k

In [ ]:
'''
def split_train_test():
  
    test = [[list(range(450 * i, 450 *(i+1))) + list(range(2250 + 450*i, 2250 + 450 * (i+1) )) + list(range(4500 + 450*i, 4500+ 450*(i+1)))]  for i in range(5)]
    train = []
    for subtest in test:
        total = list(range(6750))
        for idx in subtest:
            total.remove(idx)
            train.append(total)
        
    return zip(np.array(train), np.array(test))
'''

def split_train_test(data):
     indices = list(range(data.shape[0]))
     random.shuffle(indices)
     step = data.shape[0]//5

     for i in range(0, data.shape[0], step):
         test_indices = [j for j in indices[i: min(i+step, data.shape[0])]]
         train_indices = [j for j in indices if j not in test_indices]
         yield test_indices, train_indices

def cross_validation(dataset):
    loss = []
    for K in tqdm(range(1,50)):
        loss_K = []
        for train_idx, test_idx in split_train_test(dataset):
            
            train_data = dataset[train_idx]
            test_data = dataset[test_idx]
            
            x_train = train_data[:, :-1]
            y_train = train_data[:, -1]
            x_test = test_data[:, :-1]
            y_test = test_data[:, -1]
            
            clf = KNN(K, x_train[:100], y_train[:100])
            #import ipdb; ipdb.set_trace()

            loss_K.append(clf.evaluate(x_test, y_test))
            
        loss.append((loss_K))
    loss = np.array(loss)
    
    best_K = loss.argmin()+1
    
    return best_K

In [ ]:
import random
data = np.concatenate((train_data, test_data))
random.shuffle(data)
print(f" The optimal K is :{cross_validation(data)}")


 <u><h4>Result:</h4></u> 
=>  According to <em>cross validation</em> and the plot of the <em>loss curve</em>, we conclude that the optimal parameter $K$ for our KNN algorithm  is <b>12</b>


### (7)- MLP

The number of paramters needed to train the *MLP* is **25219** parameters where **24400** are weights and **819** biases 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(32, input_shape=(28*28,), activation="relu"))
model.add(Dense(3, activation="softmax"))

model.summary()

In [ ]:
from tensorflow.keras.utils import to_categorical

# Adapting the data
y_train[4000:6000] = 2*np.ones(2000)
y_test[500:750] = 2*np.ones(250)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Compiling the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics = ['accuracy'])
print("Training started..., this can take a while:")

#Fitting the model
model.fit(x_train, y_train, epochs = 50)

# Evaluation the model
print("Final loss value:", model.evaluate(x_test, y_test ))

<u><h3>(8)- Results of the models:</h3></u>

In [ ]:
# Loading the data
train_data, test_data = load_train(), load_test()

# Importing train and test data
x_train, y_train = train_data[:, :-1], train_data[:, -1]
x_test, y_test = test_data[:, :-1], test_data[:, -1]

In [ ]:
def zero_one_loss(x_train, y_train, x_test, y_test, classifier):
    if classifier == "Linear Regression":
        clf = LinearRegression(theta_0 = np.zeros(785), max_iter = 5000, learning_rate= 0.0001)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        
    elif classifier == "Logistic Regression":
        clf = LogisticRegression(theta_0 = np.zeros(785), max_iter = 500, learning_rate= 0.01)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
    
    elif classifier == "Perceptron":
        clf = Perceptron(theta_0 = np.zeros(785), max_iter = 10000, learning_rate= 0.001)
        clf.fit(x_train, y_train)
    
    elif classifier == "KNN":
        clf = KNN(12, x_train, y_train)
        return clf.evaluate(x_test, y_test)
    
    elif classifier == "MLP":
        y_pred = np.array(list(map(np.argmax, model.predict(x_test))))
        y_pred = y_pred % 2 
        y_pred= np.array(list(map(lambda x: x-1 if (x == 0) else x, y_pred)))

    return sum(abs(y_pred - y_test)/(2*len(y_test)))

### Linear Regression

In [ ]:
print(f"Testing Loss: {zero_one_loss(x_train, y_train, x_test, y_test, 'Linear Regression')}")
print(f"Training Loss: {zero_one_loss(x_train, y_train, x_train, y_train, 'Linear Regression')}")

### Logistic Regression

In [ ]:
print(f"Testing Loss: {zero_one_loss(x_train, y_train, x_test, y_test, 'Logistic Regression')}")
print(f"Training Loss: {zero_one_loss(x_train, y_train, x_train, y_train, 'Logistic Regression')}")

### Perceptron

In [ ]:
print(f"Testing Loss: {zero_one_loss(x_train, y_train, x_test, y_test, 'Perceptron')}")
print(f"Training Loss: {zero_one_loss(x_train, y_train, x_train, y_train, 'Perceptron')}")

### KNN

In [ ]:
print(f"Testing Loss: {zero_one_loss(x_train, y_train, x_test, y_test, 'KNN')}")
print(f"Training Loss: {zero_one_loss(x_train, y_train, x_train, y_train, 'KNN')}")

### MLP

In [ ]:
# If ValueError appears, please reload the data, it is because y_test is still one hot encoded from the training of the neural network 
print(f"Testing Loss: {zero_one_loss(x_train, y_train, x_test, y_test, 'MLP')}")
print(f"Training Loss: {zero_one_loss(x_train, y_train, x_train, y_train, 'MLP')}")

|  | Logistic Regression | OLS | Perceptron | KNN | MLP |
| --- | --- | --- | --- | --- | --- |
| Empirical Error (0-1 loss) | 0.044 | 0.121|  0.0518 |0.0385   | 0.0039|
| Test Error (0-1 loss) |  0.0624|  0.132| 0.0986 | 0.0506 |0.038 |


<u><h3> Comment on the results :</h3></u> 

* The best algorithm for our classification problem is **MLP** with a test error of **0.038**, which is expected thanks to its ability to model non-linear relationships, which is not the case for **GLMs** (General Linear Models) like <b>Logistic Regression, Linear Regression</b> and the **Perceptron Algorithm**.


* The least performing algorithm is **KNN**, with a test error of **0.424** (slightly better than the random algorihtm), which was expected since KNN only compares two images by taking into account the differences in the pixels' intensity.

### (10-) Regularization of logistic regression

#### (a) -Test and training classification errors with the 0-1 loss associated with $\hat{\theta}_\lambda$ as a function of $\lambda$.

In [ ]:
def test_train_lambda(lambdas, norm='l2'):
    clf = LogisticRegression(theta_0 = np.zeros(785), max_iter = 10, learning_rate= 0.01)
    train_loss , test_loss = [], []
    
    if norm == "l1":
        for lam in lambdas:
            clf.fit_reg_l1(x_train, y_train, lam)
            train_loss.append(clf.zero_one_loss(x_train, y_train))
            test_loss.append(clf.zero_one_loss(x_test, y_test))
    elif norm == "l2":
        for lam in lambdas:
            clf.fit_reg_l1(x_train, y_train, lam)
            train_loss.append(clf.zero_one_loss(x_train, y_train))
            test_loss.append(clf.zero_one_loss(x_test, y_test))
            
    # plotting the curves
    plt.plot(lambdas, train_loss, label="Training Loss")
    plt.plot(lambdas, test_loss, label="Test Loss")
    
    plt.title(f"Training and Test Loss as functions of lambda for {norm} regularization")
    plt.legend()
    plt.show()
    
    return train_loss, test_loss

In [ ]:
lambdas = np.linspace(0,0.0005,10)
losses_l2  = test_train_lambda(lambdas, "l2")

In [ ]:
lambdas = np.linspace(0,0.0005,10)
losses_l1  = test_train_lambda(lambdas, "l1")

#### (b)- Best value for $\lambda$ 

In [ ]:
lambdas[np.argmin(losses_l2)]

=> From the plots above, we deduce that the best value for lambda for the $l2$ norm, is arround $0.00013$

In [ ]:
lambdas[np.argmin(losses_l1)]

=> From the plots above, we deduce that the best value for lambda for the $l1$ norm, is arround $0.00013$

#### (b)- Images of the estimator $\hat{\theta}_\lambda$ 

In [ ]:
four_lambdas = [ 0.0001, 0.00015, 0.0002, 0.00025]

thetas_1 = []
thetas_2 = []

for lam in four_lambdas:
    clf_l1 = LogisticRegression(theta_0 = np.zeros(785), max_iter = 5000, learning_rate= 0.001)
    clf_l1.fit_reg_l1(x_train, y_train, lam)
    thetas_1.append(clf_l1.theta)
    
    clf_l2 = LogisticRegression(theta_0 = np.zeros(785), max_iter = 5000, learning_rate= 0.001)
    clf_l2.fit_reg_l2(x_train, y_train, lam)
    thetas_2.append(clf_l2.theta)
    

#### Images of the estimator $\hat{\theta}_\lambda$ for the $l2$ norm

In [ ]:
show_thetas(thetas_2)

#### Images of the estimator $\hat{\theta}_\lambda$ for the $l1$ norm

In [ ]:
show_thetas(thetas_1)